In [1]:
import imaplib
import email
from email.header import decode_header
import base64
from bs4 import BeautifulSoup
import re
import html2text

In [2]:
mail_pass = "7PP4ZDXqnv"
username = "lazutkina@cs.vsu.ru"
imap_server = "info.vsu.ru"
imap = imaplib.IMAP4_SSL(imap_server)
imap.login(username, mail_pass)

('OK', [b'completed'])

In [4]:
imap.list()
imap.select("inbox")
result, data = imap.search(None, "ALL")

ids = data[0]
id_list = ids.split()
latest_email_id = id_list[-1]

result, data = imap.fetch(latest_email_id, "(RFC822)")
raw_email = data[0][1]
raw_email_string = raw_email.decode('utf-8')
raw_email_string


'ARC-Message-Signature: i=2; a=rsa-sha256; c=relaxed/relaxed; d=cs.vsu.ru;\r\n\ts=20210314; t=1715162554;\r\n\th=from:from:reply-to:subject:subject:date:date:message-id:message-id:\r\n\t to:to:cc:mime-version:mime-version:content-type:content-type:\r\n\t in-reply-to:in-reply-to:references:references:dkim-signature;\r\n\tbh=1GXrt+xE/rf1A6BdAayoXKrIE1cEQdx/PcMfof88g58=;\r\n\tb=rOuQfVwfob47xHorTmmWtoRKlp8TxkF6JQBDm0sPQ/42ZBaAMMPzLmW7m29IWL+pcHIbHg\r\n\t5dwFJJNAyIy7ggUtVj+dHfcYMOZfvYkTyXOZGRfBrLcXAwf/Mwx7qM9ThjyIf/tZk2mvs3\r\n\tXvfNDilmgXXprIIOMjZZUyYT6Z/XTc0=\r\nARC-Authentication-Results: i=2;\r\n\trelay1.vsu.ru;\r\n\tdkim=pass header.d=outlook.com header.s=selector1 header.b=hQjByfic;\r\n\tdmarc=pass (policy=none) header.from=outlook.com;\r\n\tspf=pass (relay1.vsu.ru: domain of testuser123461@outlook.com designates 2a01:111:f403:2e07::800 as permitted sender) smtp.mailfrom=testuser123461@outlook.com;\r\n\tarc=pass ("microsoft.com:s=arcselector9901:i=1")\r\nARC-Seal: i=2; s=20210314; d=c

In [5]:
email_message = email.message_from_string(raw_email_string)
email_message

In [6]:
maintype = email_message.get_content_maintype()
maintype

'multipart'

In [88]:
from imap_tools import MailBox
from imap_tools import AND

password = "7PP4ZDXqnv"
username = "lazutkina@cs.vsu.ru"
host = "info.vsu.ru"

mailbox = MailBox(host)
mailbox.login(username=username, password=password, initial_folder='INBOX') 


In [12]:
# LIST: get all subfolders of the specified folder (root by default)
for f in mailbox.folder.list('INBOX'):
    print(f)  # FolderInfo(name='INBOX|cats', delim='|', flags=('\\Unmarked', '\\HasChildren'))
    print(f.name)



FolderInfo(name='INBOX/Заявки', delim='/', flags=('\\HasNoChildren', '\\UnMarked'))
INBOX/Заявки
FolderInfo(name='INBOX/К пересылке', delim='/', flags=('\\HasNoChildren', '\\UnMarked'))
INBOX/К пересылке
FolderInfo(name='INBOX/Старые', delim='/', flags=('\\HasNoChildren', '\\UnMarked'))
INBOX/Старые
FolderInfo(name='INBOX/тест', delim='/', flags=('\\HasNoChildren', '\\UnMarked'))
INBOX/тест


In [89]:
import pandas as pd


def login_mailbox(host, username, password):
    mailbox = MailBox(host)
    mailbox.login(username=username, password=password)
    return mailbox


def mails_to_df(mails):
    columns = ['id', 'subject', 'from', 'text', 'date', 'attachment_filename', 'attachment_content_type']
    mails_list = []
    for msg in mails:
        attachment_filename = ",".join(str(att.filename) for att in msg.attachments)
        attachment_content_type = ",".join(str(att.content_type) for att in msg.attachments)
        mails_list.append([msg.uid, msg.subject, msg.from_, msg.text, msg.date_str,
                           attachment_filename, attachment_content_type])

    df = pd.DataFrame(data=mails_list)
    df.columns = columns
    return df


In [90]:
password="7PP4ZDXqnv"
username="lazutkina@cs.vsu.ru"
host="info.vsu.ru"
email_folder_name='INBOX/тест'

mailbox = login_mailbox(host, username, password)
mailbox.folder.set(email_folder_name)
mails = mailbox.fetch()
df = mails_to_df(mails)
df

,id,subject,from,text,date,attachment_filename,attachment_content_type
0,1,Fwd: Тема1,0864213@inbox.ru,\n\n\n\n-------- Пересылаемое сообщение ------...,"Sat, 11 May 2024 20:48:04 +0300",,
1,2,Fwd: Тема2,0864213@inbox.ru,\n\n\n\n-------- Пересылаемое сообщение ------...,"Sat, 11 May 2024 20:48:22 +0300",Лазуткина_группа3_вариант9.pdf,application/pdf
2,3,Fwd:,0864213@inbox.ru,\n\n\n\n-------- Пересылаемое сообщение ------...,"Sat, 11 May 2024 20:48:33 +0300",,
3,4,Fwd:,0864213@inbox.ru,\n\n\n\n-------- Пересылаемое сообщение ------...,"Sat, 11 May 2024 20:48:44 +0300",Лазуткина_группа3_вариант9.pdf,application/pdf
4,5,Fwd:,0864213@inbox.ru,\n\n\n\n-------- Пересылаемое сообщение ------...,"Sat, 11 May 2024 20:48:59 +0300",Лазуткина_группа3_вариант9.pdf,application/pdf
5,6,ntvf,0864213@inbox.ru,\nntrc,"Sun, 12 May 2024 00:59:14 +0300",,


In [80]:
df

,id,subject,from,text,date,attachment_filename,attachment_content_type
0,1,Тема1,Иванов Иван <testuser123461@outlook.com>,\n \nСообщение1 \n---------------------------...,"Суббота, 11 мая 2024, 13:01 +03:00",,
1,2,Тема2,Иванов Иван <testuser123461@outlook.com>,\n \nСообщение2 \n---------------------------...,"Суббота, 11 мая 2024, 13:02 +03:00",Лазуткина_группа3_вариант9.pdf,application/pdf
2,3,,Иванов Иван <testuser123461@outlook.com>,\n \n \n------------------------------------...,"Суббота, 11 мая 2024, 13:02 +03:00",,
3,4,,Иванов Иван <testuser123461@outlook.com>,\n \n \n------------------------------------...,"Суббота, 11 мая 2024, 13:02 +03:00",Лазуткина_группа3_вариант9.pdf,application/pdf
4,5,,Иванов Иван <testuser123461@outlook.com>,\n \nТекст \n--------------------------------...,"Суббота, 11 мая 2024, 13:06 +03:00",Лазуткина_группа3_вариант9.pdf,application/pdf


In [92]:
def preprocess_subject(subjects: list[str]):
    column = []
    for s in subjects:
        if s:
            column.append(s.removeprefix("Fwd: "))
        else:
            column.append('Нет темы')
    return column

df['subject'] = preprocess_subject(df['subject'])
df

,id,subject,from,text,date,attachment_filename,attachment_content_type
0,1,Тема1,0864213@inbox.ru,\n\n\n\n-------- Пересылаемое сообщение ------...,"Sat, 11 May 2024 20:48:04 +0300",,
1,2,Тема2,0864213@inbox.ru,\n\n\n\n-------- Пересылаемое сообщение ------...,"Sat, 11 May 2024 20:48:22 +0300",Лазуткина_группа3_вариант9.pdf,application/pdf
2,3,Нет темы,0864213@inbox.ru,\n\n\n\n-------- Пересылаемое сообщение ------...,"Sat, 11 May 2024 20:48:33 +0300",,
3,4,Нет темы,0864213@inbox.ru,\n\n\n\n-------- Пересылаемое сообщение ------...,"Sat, 11 May 2024 20:48:44 +0300",Лазуткина_группа3_вариант9.pdf,application/pdf
4,5,Нет темы,0864213@inbox.ru,\n\n\n\n-------- Пересылаемое сообщение ------...,"Sat, 11 May 2024 20:48:59 +0300",Лазуткина_группа3_вариант9.pdf,application/pdf
5,6,ntvf,0864213@inbox.ru,\nntrc,"Sun, 12 May 2024 00:59:14 +0300",,


In [107]:
import datetime
from dateutil import parser
import dateparser
from datetime import datetime

In [108]:
def change_type(dates):
    column = []
    for date in dates:
        column.append(dateparser.parse(date).strftime("%d.%m.%Y %H:%M:%S"))
    return column


In [109]:
df['date'] = change_type(df['date'])
df

,id,subject,from,text,date,attachment_filename,attachment_content_type
0,1,Тема1,Иванов Иван <testuser123461@outlook.com>,\n \nСообщение1 \n---------------------------...,11.05.2024 13:01:00,,
1,2,Тема2,Иванов Иван <testuser123461@outlook.com>,\n \nСообщение2 \n---------------------------...,11.05.2024 13:02:00,Лазуткина_группа3_вариант9.pdf,application/pdf
2,3,Нет темы,Иванов Иван <testuser123461@outlook.com>,\n \n \n------------------------------------...,11.05.2024 13:02:00,,
3,4,Нет темы,Иванов Иван <testuser123461@outlook.com>,\n \n \n------------------------------------...,11.05.2024 13:02:00,Лазуткина_группа3_вариант9.pdf,application/pdf
4,5,Нет темы,Иванов Иван <testuser123461@outlook.com>,\n \nТекст \n--------------------------------...,11.05.2024 13:06:00,Лазуткина_группа3_вариант9.pdf,application/pdf
5,6,ntvf,0864213@inbox.ru,\nntrc,12.05.2024 00:59:14,,


In [116]:
def preprocess_text(texts):
    column = []
    for t in texts:
        # x = text.lower()
        # remove extra new lines
        t = re.sub(r'\n+', ' ', t)
        # # removing (replacing with empty spaces actually) all the punctuations
        # x = re.sub("[" + string.punctuation + "]", " ", x)
        # remove extra white spaces
        
        t = re.sub(r'-{2,}', '', t)
        t = re.sub(r'\s+', ' ', t)
        column.append(t)
        
    return column

In [117]:
df['text'] = preprocess_text(df['text'])
df

,id,subject,from,text,date,attachment_filename,attachment_content_type
0,1,Тема1,Иванов Иван <testuser123461@outlook.com>,Сообщение1 Настя Лазуткина Отправлено из Почт...,11.05.2024 13:01:00,,
1,2,Тема2,Иванов Иван <testuser123461@outlook.com>,Сообщение2 Настя Лазуткина Отправлено из Почт...,11.05.2024 13:02:00,Лазуткина_группа3_вариант9.pdf,application/pdf
2,3,Нет темы,Иванов Иван <testuser123461@outlook.com>,Настя Лазуткина Отправлено из Почты Mail.ru,11.05.2024 13:02:00,,
3,4,Нет темы,Иванов Иван <testuser123461@outlook.com>,Настя Лазуткина Отправлено из Почты Mail.ru,11.05.2024 13:02:00,Лазуткина_группа3_вариант9.pdf,application/pdf
4,5,Нет темы,Иванов Иван <testuser123461@outlook.com>,Текст Настя Лазуткина Отправлено из Почты Mai...,11.05.2024 13:06:00,Лазуткина_группа3_вариант9.pdf,application/pdf
5,6,ntvf,0864213@inbox.ru,ntrc,12.05.2024 00:59:14,,


In [93]:
def preprocess(df: pd.DataFrame):
    for index, row in df.iterrows(): 
        text = row['text']
        if text:
            if 'Пересылаемое сообщение' in text:
                sender = re.search(r'От кого: (.*?)\n', text).group(1)
                df.at[index, 'from'] = sender
                date = re.search(r'Дата: (.*?)\n', text).group(1)
                df.at[index, 'date'] = date
                subject = re.search(r'Тема: (.*?)\n', text)
                if subject is not None:
                    message = text.split(subject.group(1))[1]
                else:
                    message = text.split(date)[1]
                df.at[index, 'text'] = message
            
        else:
            df.at[index, 'text'] = 'Нет тела сообщения'
    return df

In [94]:
df

,id,subject,from,text,date,attachment_filename,attachment_content_type
0,1,Тема1,0864213@inbox.ru,\n\n\n\n-------- Пересылаемое сообщение ------...,"Sat, 11 May 2024 20:48:04 +0300",,
1,2,Тема2,0864213@inbox.ru,\n\n\n\n-------- Пересылаемое сообщение ------...,"Sat, 11 May 2024 20:48:22 +0300",Лазуткина_группа3_вариант9.pdf,application/pdf
2,3,Нет темы,0864213@inbox.ru,\n\n\n\n-------- Пересылаемое сообщение ------...,"Sat, 11 May 2024 20:48:33 +0300",,
3,4,Нет темы,0864213@inbox.ru,\n\n\n\n-------- Пересылаемое сообщение ------...,"Sat, 11 May 2024 20:48:44 +0300",Лазуткина_группа3_вариант9.pdf,application/pdf
4,5,Нет темы,0864213@inbox.ru,\n\n\n\n-------- Пересылаемое сообщение ------...,"Sat, 11 May 2024 20:48:59 +0300",Лазуткина_группа3_вариант9.pdf,application/pdf
5,6,ntvf,0864213@inbox.ru,\nntrc,"Sun, 12 May 2024 00:59:14 +0300",,


In [95]:
preprocess(df)
df

,id,subject,from,text,date,attachment_filename,attachment_content_type
0,1,Тема1,Иванов Иван <testuser123461@outlook.com>,\n \nСообщение1 \n---------------------------...,"Суббота, 11 мая 2024, 13:01 +03:00",,
1,2,Тема2,Иванов Иван <testuser123461@outlook.com>,\n \nСообщение2 \n---------------------------...,"Суббота, 11 мая 2024, 13:02 +03:00",Лазуткина_группа3_вариант9.pdf,application/pdf
2,3,Нет темы,Иванов Иван <testuser123461@outlook.com>,\n \n \n------------------------------------...,"Суббота, 11 мая 2024, 13:02 +03:00",,
3,4,Нет темы,Иванов Иван <testuser123461@outlook.com>,\n \n \n------------------------------------...,"Суббота, 11 мая 2024, 13:02 +03:00",Лазуткина_группа3_вариант9.pdf,application/pdf
4,5,Нет темы,Иванов Иван <testuser123461@outlook.com>,\n \nТекст \n--------------------------------...,"Суббота, 11 мая 2024, 13:06 +03:00",Лазуткина_группа3_вариант9.pdf,application/pdf
5,6,ntvf,0864213@inbox.ru,\nntrc,"Sun, 12 May 2024 00:59:14 +0300",,


In [67]:
preprocess(df)
df

,id,subject,from,text,date,attachment_filename,attachment_content_type
0,1,Тема1,0864213@inbox.ru,\n \nСообщение1 \n---------------------------...,"Sat, 11 May 2024 20:48:04 +0300",,
1,2,Тема2,0864213@inbox.ru,\n \nСообщение2 \n---------------------------...,"Sat, 11 May 2024 20:48:22 +0300",Лазуткина_группа3_вариант9.pdf,application/pdf
2,3,Нет темы,0864213@inbox.ru,\n \n \n------------------------------------...,"Sat, 11 May 2024 20:48:33 +0300",,
3,4,Нет темы,0864213@inbox.ru,\n \n \n------------------------------------...,"Sat, 11 May 2024 20:48:44 +0300",Лазуткина_группа3_вариант9.pdf,application/pdf
4,5,Нет темы,0864213@inbox.ru,\n \nТекст \n--------------------------------...,"Sat, 11 May 2024 20:48:59 +0300",Лазуткина_группа3_вариант9.pdf,application/pdf


In [78]:
for index, row in df.iterrows():
    print(index)
    text = row['text']
    if text:
        print('true')
        if 'Пересылаемое сообщение' in text:
            print('fwd')
            sender = re.search(r'От кого: (.*?)\n', text).group(1)
            print(sender)
            df.at[index, 'from'] = sender
            date = re.search(r'Дата: (.*?)\n', text).group(1)
            df.at[index, 'date'] = date
            subject = re.search(r'Тема: (.*?)\n', text)
            if subject is not None:
                message = text.split(subject.group(1))[1]
            else:
                message = text.split(date)[1]
            df.at[index, 'text'] = message
        
    else:
        df.at[index, 'text'] = 'Нет тела сообщения'


0
true
fwd
Иванов Иван <testuser123461@outlook.com>
1
true
fwd
Иванов Иван <testuser123461@outlook.com>
2
true
fwd
Иванов Иван <testuser123461@outlook.com>
3
true
fwd
Иванов Иван <testuser123461@outlook.com>
4
true
fwd
Иванов Иван <testuser123461@outlook.com>


In [79]:
df

,id,subject,from,text,date,attachment_filename,attachment_content_type
0,1,Тема1,Иванов Иван <testuser123461@outlook.com>,\n \nСообщение1 \n---------------------------...,"Суббота, 11 мая 2024, 13:01 +03:00",,
1,2,Тема2,Иванов Иван <testuser123461@outlook.com>,\n \nСообщение2 \n---------------------------...,"Суббота, 11 мая 2024, 13:02 +03:00",Лазуткина_группа3_вариант9.pdf,application/pdf
2,3,,Иванов Иван <testuser123461@outlook.com>,\n \n \n------------------------------------...,"Суббота, 11 мая 2024, 13:02 +03:00",,
3,4,,Иванов Иван <testuser123461@outlook.com>,\n \n \n------------------------------------...,"Суббота, 11 мая 2024, 13:02 +03:00",Лазуткина_группа3_вариант9.pdf,application/pdf
4,5,,Иванов Иван <testuser123461@outlook.com>,\n \nТекст \n--------------------------------...,"Суббота, 11 мая 2024, 13:06 +03:00",Лазуткина_группа3_вариант9.pdf,application/pdf


In [60]:
df['text']

0    \n \nСообщение1  \n---------------------------...
1    \n \nСообщение2  \n---------------------------...
2    \n \n   \n------------------------------------...
3    \n \n   \n------------------------------------...
4    \n \nТекст  \n--------------------------------...
Name: text, dtype: object

In [61]:
df['text'].iloc[0]

'\n\xa0\nСообщение1 \xa0\n----------------------------------------------------------------------\n\xa0\n--\nНастя Лазуткина\nОтправлено из Почты  Mail.ru\n\xa0'